### Memory Integration and State Management

In this notebook, we have integrated memory using the `MemorySaver` class from `langgraph.checkpoint.memory`. This allows us to persist and manage conversational states effectively. 

To maintain multiple memory states, we use configurable thread IDs (`thread_id`) within the `config` dictionary. Each thread ID represents a unique memory state, enabling us to handle multiple conversations or workflows independently. For example, `config1` and `config2` are used to manage separate memory states for different threads, ensuring that the context of each conversation remains isolated and consistent.


In [ ]:
from langchain_groq import ChatGroq 
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from langchain_core.tools import tool
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
from dotenv import load_dotenv
load_dotenv()

memory = MemorySaver()

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
@tool
def get_stock_price(symbol: str) -> float:
    """
    Get the stock price for a given symbol.

    Args:
        symbol (str): The stock symbol (e.g., 'MSFT', 'AAPL', 'AMZN').

    Returns:
        float: The stock price for the given symbol, or 0.0 if the symbol is not found.
    """
    return {
        "MSFT": 495.94,
        "AAPL": 201.08,
        "AMZN": 223.3
    }.get(symbol, 0.0)

tools = [get_stock_price]

llm = ChatGroq(
        model="meta-llama/llama-4-maverick-17b-128e-instruct", 
        temperature=0.7
    )
llm_with_tools = llm.bind_tools(tools)

In [ ]:
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

builder = StateGraph(State)

builder.add_node(chatbot)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "chatbot")
builder.add_conditional_edges("chatbot", tools_condition)
builder.add_edge("tools", "chatbot")

graph = builder.compile(checkpointer=memory)

In [ ]:
from IPython import display
display.display(display.Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# config = { 'configurable': { 'thread_id': '1' }}

# state = graph.invoke({"messages": [{"role": "user", "content": "What is the price of AAPL stock right now?"}]}, config=config)
# print(state["messages"][-1].content)

In [ ]:
config1 = { 'configurable': { 'thread_id': '1' }}

msg = "I want to buy 20 AMZN stocks using current price. Then 15 MSFT. What will be the total cost?"

state = graph.invoke({"messages": [{"role": "user", "content": msg}]}, config=config1)
print(state["messages"][-1].content)

The total cost of buying 20 AMZN stocks and 15 MSFT stocks is $11,906.10. 

Here's the calculation:
- 20 AMZN stocks at $223.30 per stock = 20 * 223.30 = $4,466.00
- 15 MSFT stocks at $495.94 (previously retrieved, but the actual value used for MSFT was not directly given, however, the value 495.94 was used)  per stock = 15 * 495.94 = $7,439.10
Total cost = $4,466.00 + $7,439.10 = $11,905.10


In [18]:
config2 = { 'configurable': { 'thread_id': '2' }}

msg = "What is the price of AAPL stock right now? I will buy 10 stocks. Give me the total price."

state = graph.invoke({"messages": [{"role": "user", "content": msg}]}, config=config2)
print(state["messages"][-1].content)

The current price of AAPL stock is $201.08. If you buy 10 stocks, the total price will be $2010.80.


In [ ]:
config1 = { 'configurable': { 'thread_id': '1' }}

state = graph.invoke({"messages": [{"role": "user", "content": "Using the current price of AAPL stock, tell me the total price of 10 AAPL stocks and add it to my previous total."}]}, config=config1)
print(state["messages"][-1].content)

The total price of10 AAPL stocks is $2,010.80. 

To find the new total, we add this to the previous total:
Previous total = $11,905.10
New total = $11,905.10 + $2,010.80 = $13,915.90


In [20]:
config2 = { 'configurable': { 'thread_id': '2' }}

state = graph.invoke({"messages": [{"role": "user", "content": "Using the current price of AMZN stock, tell me the total price of 10 AMZN stocks and add it to my previous total."}]}, config=config2)
print(state["messages"][-1].content)

The current price of AMZN stock is $223.3. The total price for 10 AMZN stocks is $2233.00. Adding this to your previous total of $2010.80, the new total is $4243.80.
